In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import random
from d2l import torch as d2l

d:\Anaconda\envs\d2l\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
### Train parameters
num_epochs = 20
batch_size = 32
num_steps = 9
learning_rate = 0.0005

In [3]:
data = d2l.MTFraEng(batch_size)

In [4]:
train_iter = data.get_dataloader(train = True)
validataion_iter = data.get_dataloader(train = False)

src_vocab = data.src_vocab
tgt_vocab = data.tgt_vocab

In [5]:
for batch in train_iter:
    print(src_vocab.to_tokens(batch[0][0].tolist()))
    # break

['go', 'home', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['be', 'still', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['i', '<unk>', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['fix', 'this', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['can', 'we', 'go', '?', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['watch', 'us', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<unk>', 'up', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['may', 'i', 'go', '?', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['i', 'swore', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['get', 'out', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
["i'm", 'wet', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
["i'm", 'calm', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
["i'm", '<unk>', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['go', 'away', '.', '<eos>', '<pad>', '<

In [6]:
for batch in train_iter:
    print(tgt_vocab.to_tokens(batch[1][0].tolist()))
    # break

['<bos>', '<unk>', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', 'courez', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', 'va', 'au', '<unk>', '!', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', 'je', 'le', 'refuse', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', 'rentre', 'chez', '<unk>', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', 'j’ai', '<unk>', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', '<unk>', 'moi', 'qui', 'ai', 'gagné', '?', '<eos>', '<pad>']
['<bos>', 'tom', 'a', 'payé', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', 'je', 'suis', '<unk>', '!', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', 'viens', 'ici', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', 'je', '<unk>', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', '<unk>', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', '<unk>', 'ceci', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', '<unk>', 'c

In [7]:
import torch
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
print(device)

2.4.0+cu121
cuda


In [8]:
class Encoder(nn.Module, d2l.HyperParameters):
    def __init__(self, inputs_size, embed_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.save_hyperparameters()

        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(inputs_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers)

    def forward(self, X):
        # X shape: (batch_size, num_steps) 
        embedding = self.dropout(self.embedding(X.T))
        # embedding shape (num_steps, batch_size, embed_size)
        outputs, (hidden, cell) = self.rnn(embedding)
        return hidden, cell

In [9]:
class Decoder(nn.Module, d2l.HyperParameters):
    def __init__(self, inputs_size, embed_size, hidden_size, output_size, num_layers, dropout):
        super().__init__()
        self.save_hyperparameters()

        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(inputs_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X, hidden, cell):
        # Shape of X: (batch_size)
        X = X.unsqueeze(0)
        # -> shape of X: (1, batch_size)

        embedding = self.dropout(self.embedding(X))
        # shape of embedding: (1, batch_size, embed_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # shape of output = (1, batch_size, hidden_size)

        preds = self.fc(outputs)
        # shape of preds = (1, batch_size, output_size)
        preds = preds.squeeze(0)

        return preds, hidden, cell

In [10]:
class Seq2Seq(nn.Module, d2l.HyperParameters):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.save_hyperparameters()

    def forward(self, source, target, teacher_force_ratio = 0):
        # source shape: (batch_size, num_steps)
        batch_size = source.shape[0]
        # num_steps = source.shape[1]
        target_vocab_size = len(data.tgt_vocab)
        target_len = target.shape[1]

        outputs = torch.zeros((target_len, batch_size, target_vocab_size), device=device)
        hidden, cell = self.encoder(source)

        target = target.T
        # shape of target (target_len, batch_size)
        x = target[0]
        
        for t in range(1, target_len):
            
            x, hidden, cell = self.decoder(x, hidden, cell)
            # print(x.shape)
            outputs[t] = x

            # x shape: (batch_size, target_vocab_size)
            best_guess = x.argmax(1) 

            x = target[t] if random.random() < teacher_force_ratio else best_guess

        # shape of model outputs: (batch_size, num_steps, len(data.tgt_vocab))
        return outputs.swapaxes(0, 1)

In [11]:
### Train parameters
# num_epochs = 100
# batch_size = 32
# num_steps = 9
# learning_rate = 0.001

# Model hyperparameters
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_input_size = len(data.src_vocab)
decoder_input_size = len(data.tgt_vocab)
encoder_embed_size, decoder_embed_size = 300, 300
encoder_hidden_size, decoder_hidden_size = 1024, 1024
encoder_num_layers = decoder_num_layers = 2
encoder_dropout = 0
decoder_dropout = 0

In [12]:
encoder = Encoder(encoder_input_size, encoder_embed_size, encoder_hidden_size, encoder_num_layers, encoder_dropout)
decoder = Decoder(decoder_input_size, decoder_embed_size, decoder_hidden_size, len(data.tgt_vocab), decoder_num_layers, decoder_dropout)

model = Seq2Seq(encoder, decoder)
model = model.to(device)

In [21]:
X = torch.zeros(size = (32, num_steps), dtype = torch.int, device = device)
hidden, cell = encoder(X)
print(hidden.shape, cell.shape)
# (num_layer, batch_size, hidden_size)

torch.Size([2, 32, 1024]) torch.Size([2, 32, 1024])


In [22]:
for X, y, _, _ in train_iter:
    print(X.shape)
    print(y.shape)
    print(model(X.to(device), y.to(device)).device)
    break

torch.Size([32, 9])
torch.Size([32, 9])
cuda:0


In [66]:
# from torch.utils.tensorboard import SummaryWriter

# writer = SummaryWriter(f"runs/Loss_plot")
# step = 0

In [67]:
pad_idx = data.tgt_vocab.token_to_idx['<pad>']
print(pad_idx)

loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

5


In [68]:
def save_checkpoint(state, filename="seq2seq_TM.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [69]:
if load_model:
    load_checkpoint(torch.load("seq2seq_TM.pth.tar"), model, optimizer)

In [70]:
def predict(src_sentence, num_preds):
    list_words = src_sentence.split()
    list_words.append("<eos>")
    print(list_words)
    list_tokens = [data.src_vocab.token_to_idx[word] for word in list_words]
    
    X = torch.tensor(list_tokens, device = device)
    X = X.unsqueeze(0)
    
    with torch.no_grad():
        hidden, cell = model.encoder(X)
    X = torch.tensor(data.tgt_vocab.token_to_idx['<bos>'], device = device).reshape(1)
    
    result = []
    for _ in range(num_preds):
        with torch.no_grad():
            output, hidden, cell = model.decoder(X, hidden, cell)
            X = output.argmax(1)
        result.append(output.argmax(1).cpu().numpy()[0])

    print(data.tgt_vocab.to_tokens(result))


predict("marry me", 10)

['marry', 'me', '<eos>']
['fainéant', 'oublie-le', 'faire', 'faire', 'faire', 'réveille-toi', 'attendez', 'attendez', 'ont', 'réveille-toi']


In [71]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1} / {num_epochs}")
    checkpoint = {'state_dict' : model.state_dict(), 'optimizer': optimizer.state_dict()}
    # save_checkpoint(checkpoint)

    for batch_idx, batch in enumerate(train_iter):
        # batch = batch.to(device)
        X, y, _, _ = batch
        X = X.to(device)
        y = y.to(device)
        model.train()
        output = model(X, y)
        # output shape: (batch_size, num_steps, len(data.tgt_vocab))
        output = output[1:].reshape(-1, output.shape[-1])
        y = y[1:].reshape(-1, )
        l = loss_fn(output, y)
        optimizer.zero_grad()
        l.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

    print("Loss: ", l) 
    print(predict("marry me", 10))
        

Epoch: 1 / 20
Loss:  tensor(3.3360, device='cuda:0', grad_fn=<NllLossBackward0>)
['marry', 'me', '<eos>']
['<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
None
Epoch: 2 / 20
Loss:  tensor(3.3417, device='cuda:0', grad_fn=<NllLossBackward0>)
['marry', 'me', '<eos>']
['<unk>', '.', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
None
Epoch: 3 / 20
Loss:  tensor(3.3008, device='cuda:0', grad_fn=<NllLossBackward0>)
['marry', 'me', '<eos>']
['sérieux', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
None
Epoch: 4 / 20
Loss:  tensor(2.9587, device='cuda:0', grad_fn=<NllLossBackward0>)
['marry', 'me', '<eos>']
['sérieux', 'sérieux', 'sérieux', 'sérieux', 'sérieux', '<unk>', '!', '<eos>', '<eos>', '<eos>']
None
Epoch: 5 / 20
Loss:  tensor(2.7434, device='cuda:0', grad_fn=<NllLossBackward0>)
['marry', 'me', '<eos>']
['sérieux', 'sérieux', 'sérieux', 'sérieux', 'sérieux', 'sérieux', 'sois', '<eos>'

In [77]:
predict("i am", 10)

['i', 'am', '<eos>']
["j'ai", '<unk>', '.', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
